In [7]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.wcs import WCS
import os
import csv

########## Input ##########

#fits_path = 'F:\\data\\spectral_fits_gold_big\\'
fits_path = os.path.abspath("./data_spectra")

samples_per_class = 2500

smallest_wavelength = 4000 # in Angström
biggest_wavelength = 9000 

########## Program ##########

all_flux = []
filenames = []

for directory in os.listdir(fits_path):
    count_failed=0
    count_added=0
    
    for filename in os.listdir(fits_path + '/' + directory):
        #path = fits_path + '\\' + directory + '\\' + filename
        path = os.path.join(fits_path, directory, filename)

        # fits-Dateien öffnen und wavelength + flux einlesen
        hdul = fits.open(path)
        data = hdul[1].data
        flux = data['flux']
        wavelength = 10**data['loglam']
        hdul.close()
        
        # first und last Index finden
        for i in range(len(wavelength)):
            if wavelength[i]>smallest_wavelength:
                first_index = i
                break

        for i in range(len(wavelength)):
            if wavelength[i]>biggest_wavelength:
                last_index = i
                break

        # wavenlength und flux Listen schneiden
        wavelength = wavelength[first_index:last_index]
        flux = flux[first_index:last_index]
        
        if len(wavelength) == 3522 and count_added < samples_per_class:
            all_flux.append(flux)
            filenames.append(filename)
            count_added += 1        

        if len(wavelength) != 3522:
            print("Länge der Liste wavelength ist: " + str(len(wavelength)))
            count_failed += 1            
            
    print(str(count_failed/samples_per_class*100) + "% waren nicht erfolgreich bei der Klasse:" + directory)

# Numpy Arrays mit Daten füllen
data = np.array(all_flux)

labels = np.zeros(shape=(4*samples_per_class,), dtype='int')
for i in range(4):
    for t in range(samples_per_class):
        labels[i*samples_per_class+t] = i
        
wavelengths = np.array(wavelength)
filenames = np.array(filenames)
# Numpy arrays in .npy Dateien speichern
np.save(fits_path +'/'+ "data.npy", data)
np.save(fits_path +'/'+ "labels.npy", labels)
np.save(fits_path +'/'+ "wavelengths.npy", wavelengths)
np.save(fits_path +'/'+ "filenames.npy", filenames)

0.0% waren nicht erfolgreich bei der Klasse:AGN
0.0% waren nicht erfolgreich bei der Klasse:galaxy
0.0% waren nicht erfolgreich bei der Klasse:QSO
0.0% waren nicht erfolgreich bei der Klasse:star
